# ELT Project

## Finding Data:

DATA SOURCE: https://www.kaggle.com/datasnaek/youtube-new/data <br/>
Utilizing: <br/>
3 csv files (Canada, US, and Mexico) <br/>
3 json file (Canada, US, and Mexico) <br/>

## Data Cleanup & Analysis

Plan and document the following:
* The sources of data that you will extract from.
* The type of transformation needed for this data (cleaning, joining, filtering, aggregating, etc).
* The type of final production database to load the data into (relational or non-relational).
* The final tables or collections that will be used in the production database.

You will be required to submit a final technical report with the above information and steps required to reproduce your ETL process.

## Project Report:

Submit a Final Report that describes the following:
* Extract: your original data sources and how the data was formatted (CSV, JSON, pgAdmin 4, etc).
* Transform: what data cleaning or transformation was required.
* Load: the final database, tables/collections, and why this was chosen.

Please upload the report to Github and submit a link to Bootcampspot.

In [1]:
#dependencies 
import os
import pandas as pd
import json
from sqlalchemy import create_engine
from bs4 import BeautifulSoup 

In [2]:
#reading in Canada csv video info
ca_file= "data/CAvideos.csv"
CA_df =pd.read_csv(ca_file)
CA_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


In [3]:
#adding a column to define which country info came from
CA_df.insert(1,"country", "CA") 
CA_df.head()

,video_id,country,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,CA,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,CA,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,CA,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,CA,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


In [4]:
#reading in Great Britain csv video info
gb_file= "data/GBvideos.csv"
GB_df =pd.read_csv(gb_file)
GB_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...


In [5]:
#adding a column to define which country info came from
GB_df.insert(1,"country", "GB")
GB_df.head()

,video_id,country,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,GB,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,GB,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,GB,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,GB,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,GB,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...


In [12]:
#reading in United States csv video info
us_file= "data/USvideos.csv"
US_df =pd.read_csv(us_file)
US_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [13]:
#adding a column to define which country info came from
US_df.insert(1,"country", "US")
US_df.head()

,video_id,country,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,US,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,US,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,US,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,US,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,US,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [15]:
#merging Canada and Great Britain
merge = CA_df.merge(GB_df, on='video_id')
merge.head()

,video_id,country_x,trending_date_x,title_x,channel_title_x,category_id_x,publish_time_x,tags_x,views_x,likes_x,...,tags_y,views_y,likes_y,dislikes_y,comment_count_y,thumbnail_link_y,comments_disabled_y,ratings_disabled_y,video_error_or_removed_y,description_y
0,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",20539417,840642,47715,124236,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
2,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",22702386,869304,50018,123227,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",24578152,891283,51977,125444,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
4,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",26448434,911883,53873,127481,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...


In [16]:
#mering us and other two countries
countries_merged = merge.merge(US_df, on ='video_id')
countries_merged.head()

,video_id,country_x,trending_date_x,title_x,channel_title_x,category_id_x,publish_time_x,tags_x,views_x,likes_x,...,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158531,787419,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",20539417,840642,47715,124236,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
2,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",22702386,869304,50018,123227,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",24578152,891282,51977,125444,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
4,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",26448434,911883,53873,127481,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...


In [17]:
#viewing all merged columns to weed out the ones needing to drop
countries_merged.columns

Index(['video_id', 'country_x', 'trending_date_x', 'title_x',
       'channel_title_x', 'category_id_x', 'publish_time_x', 'tags_x',
       'views_x', 'likes_x', 'dislikes_x', 'comment_count_x',
       'thumbnail_link_x', 'comments_disabled_x', 'ratings_disabled_x',
       'video_error_or_removed_x', 'description_x', 'country_y',
       'trending_date_y', 'title_y', 'channel_title_y', 'category_id_y',
       'publish_time_y', 'tags_y', 'views_y', 'likes_y', 'dislikes_y',
       'comment_count_y', 'thumbnail_link_y', 'comments_disabled_y',
       'ratings_disabled_y', 'video_error_or_removed_y', 'description_y',
       'country', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [18]:
#drop unnecessary columns not using for project
countries_merged.drop(['tags_x', 'comment_count_x', 'thumbnail_link_x', 'comments_disabled_x', 'ratings_disabled_x','video_error_or_removed_x', 'tags_y', 'comment_count_y', 'thumbnail_link_y', 'comments_disabled_y', 'ratings_disabled_y', 'video_error_or_removed_y',  'tags', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled','video_error_or_removed','publish_time_x','publish_time_y','publish_time'] , axis=1, inplace=True)
countries_merged.head()

,video_id,country_x,trending_date_x,title_x,channel_title_x,category_id_x,views_x,likes_x,dislikes_x,description_x,...,description_y,country,trending_date,title,channel_title,category_id,views,likes,dislikes,description
0,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158531,787419,43420,Eminem's new track Walk on Water ft. Beyoncé i...
1,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.15.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,20539417,840642,47715,Eminem's new track Walk on Water ft. Beyoncé i...
2,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.16.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,22702386,869304,50018,Eminem's new track Walk on Water ft. Beyoncé i...
3,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.17.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,24578152,891282,51977,Eminem's new track Walk on Water ft. Beyoncé i...
4,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.18.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,26448434,911883,53873,Eminem's new track Walk on Water ft. Beyoncé i...


In [19]:
#seeing all coulmns in left merged df
countries_merged.columns

Index(['video_id', 'country_x', 'trending_date_x', 'title_x',
       'channel_title_x', 'category_id_x', 'views_x', 'likes_x', 'dislikes_x',
       'description_x', 'country_y', 'trending_date_y', 'title_y',
       'channel_title_y', 'category_id_y', 'views_y', 'likes_y', 'dislikes_y',
       'description_y', 'country', 'trending_date', 'title', 'channel_title',
       'category_id', 'views', 'likes', 'dislikes', 'description'],
      dtype='object')

In [48]:
#renaming columns by countries
#these are the coloumns to be renamed
# 'video_id', 
# CA= 'country_x', 'trending_date_x', 'title_x','channel_title_x', 'category_id_x','views_x', 'likes_x', 'dislikes_x', 'description_x', 
# GB = 'country_y','trending_date_y', 'title_y', 'channel_title_y', 'category_id_y', 'description_y','views_y', 'likes_y', 'dislikes_y',
# US = 'country', 'trending_date', 'title', 'channel_title', 'category_id','views', 'likes', 'dislikes','description'

countries_df=countries_merged.rename(columns={
    'country_x':'Canada','trending_date_x':'trend_date_CA','title_x':'title_CA','channel_title_x':'channel_title_CA','category_id_x':'category_id_CA','views_x':'views_CA', 'likes_x':'likes_CA', 'dislikes_x': 'dislikes_CA', 'description_x':'discriptions_CA',
    'country_y':'Great Britain','trending_date_y':'trend_date_GB', 'title_y':'title_GB', 'channel_title_y':'channel_title_GB', 'category_id_y':'category_id_GB','views_y':'views_GB','likes_y':'likes_GB', 'dislikes_y':'dislikes_GB','description_y':'discriptions_GB',
    'country':'United States', 'trending_date': 'tend_date_US', 'title':'title_US', 'channel_title':'channel_title_US', 'category_id':'category_id_US','views':'views_US', 'likes':'likes_US', 'dislikes':'dislikes_US','description':'description_US'})
countries_df.head()

,video_id,Canada,trend_date_CA,title_CA,channel_title_CA,category_id_CA,views_CA,likes_CA,dislikes_CA,discriptions_CA,...,discriptions_GB,United States,tend_date_US,title_US,channel_title_US,category_id_US,views_US,likes_US,dislikes_US,description_US
0,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158531,787419,43420,Eminem's new track Walk on Water ft. Beyoncé i...
1,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.15.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,20539417,840642,47715,Eminem's new track Walk on Water ft. Beyoncé i...
2,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.16.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,22702386,869304,50018,Eminem's new track Walk on Water ft. Beyoncé i...
3,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.17.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,24578152,891282,51977,Eminem's new track Walk on Water ft. Beyoncé i...
4,n1WpP7iowLc,CA,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,17158579,787425,43420,Eminem's new track Walk on Water ft. Beyoncé i...,...,Eminem's new track Walk on Water ft. Beyoncé i...,US,17.18.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,26448434,911883,53873,Eminem's new track Walk on Water ft. Beyoncé i...


In [58]:
countries_df.columns

Index(['video_id', 'Canada', 'trend_date_CA', 'title_CA', 'channel_title_CA',
       'category_id_CA', 'views_CA', 'likes_CA', 'dislikes_CA',
       'discriptions_CA', 'Great Britain', 'trend_date_GB', 'title_GB',
       'channel_title_GB', 'category_id_GB', 'views_GB', 'likes_GB',
       'dislikes_GB', 'discriptions_GB', 'United States', 'tend_date_US',
       'title_US', 'channel_title_US', 'category_id_US', 'views_US',
       'likes_US', 'dislikes_US', 'description_US'],
      dtype='object')

In [56]:
connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [57]:
# Confirm tables
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/e3q8)